# Setup

## Importing and testing Environment

### Importing

In [1]:
#import shutup; shutup.please()
import logging    
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import numpy as np

np.finfo(np.dtype("float32"))
np.finfo(np.dtype("float64"))

import cv2
import tensorflow as tf 
from tensorflow import keras
#tf.config.set_per_process_memory_growth(True)

import pickle

import collections
from collections import defaultdict
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, LeakyReLU, Dropout, Flatten, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Embedding, Reshape, Activation
from keras.layers import Concatenate, Conv2DTranspose, multiply, UpSampling2D
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.utils import Progbar

np.random.seed(16)
tf.random.set_seed(16)

2023-02-06 19:56:17.289718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 19:56:17.496117: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Silencing TF

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

### Test OpenCV2 Cuda-Enabled

In [4]:
cv2.cuda.getDevice()

0

In [5]:
cv2.cuda.getCudaEnabledDeviceCount()

1

### Test TensorFlow CUDA-Enabled

In [3]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("Num CPUs Available: ", len(tf.config.list_logical_devices('GPU')))
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num CPUs Available:  1
Num CPUs Available:  1
Num GPUs Available:  1


In [3]:
print(tf.__version__)

2.11.0


In [5]:
print(tf.test.is_built_with_cuda())
# True or False
print(tf.test.is_built_with_gpu_support())
# True or False
print(tf.test.is_built_with_rocm())
# True or False
print(tf.test.is_built_with_xla())
# True or False
print(tf.config.list_physical_devices(device_type='GPU'))
print(tf.config.list_logical_devices(device_type='GPU'))

True
True
False
True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[LogicalDevice(name='/device:GPU:0', device_type='GPU')]


# Data Loader

In [7]:
os.getcwd()

'/home/giorgio/Projects_Linux/chest-xrays-classification-generation'

In [2]:
# set batch size and image size
batch_size=36
img_size=112

# set training, validation, and testing paths
training_path = os.path.join(f'./Data/COVIDx-splitted-resized-{img_size}/train')
val_path = os.path.join(f'./Data/COVIDx-splitted-resized-{img_size}/val')
testing_path = os.path.join(f'./Data/COVIDx-splitted-resized-{img_size}/test')

# initialize training data generator with validation split and data augmentation
train_datagen = ImageDataGenerator(validation_split = 0.1, shear_range=0.1, zoom_range=0.1)
# initialize testing data generator with data augmentation
test_datagen = ImageDataGenerator(shear_range=0.1, zoom_range=0.1)

# generate training data from training path with batch size and target size specified
train_data = train_datagen.flow_from_directory(training_path, subset='training', batch_size = batch_size, target_size = (img_size, img_size),
                                               shuffle = True, class_mode = 'binary', seed = 42)

# generate validation data from training path with batch size and target size specified
val_data = train_datagen.flow_from_directory(training_path, subset='validation', batch_size = batch_size, target_size = (img_size, img_size), 
                                             shuffle = True, class_mode = 'binary', seed = 42) 

# generate testing data from testing path with batch size and target size specified
test_data = test_datagen.flow_from_directory(testing_path, batch_size = batch_size, target_size = (img_size, img_size),
                                             shuffle = True, class_mode = 'binary', seed = 42)

# print class indices for binary classification
print(train_data.class_indices)
print(val_data.class_indices)
print(test_data.class_indices)


# get the first batch of training data
batchX, batchy = train_data.next()
# print the shape of training batch and label batch
print('train_batch.shape:', batchX.shape)
print('label_batch.shape', batchy.shape)

Found 26465 images belonging to 3 classes.
Found 2939 images belonging to 3 classes.
Found 400 images belonging to 3 classes.
{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
{'COVID-19': 0, 'normal': 1, 'pneumonia': 2}
train_batch.shape: (36, 112, 112, 3)
label_batch.shape (36,)


In [ ]:
batchX[1]

# Discriminator Model 1 

In [3]:
def discriminator(input_shape=(112, 112, 3)):
    #weight initialization
    init = RandomNormal(mean = 0.0, stddev = 0.02)
    
    # convolutional block
    def conv_block(input_layer, filter_size, stride):
        x = Conv2D(filter_size, kernel_size = (3,3), padding='same', 
                   strides=stride, kernel_initializer = init)(input_layer)
        x = BatchNormalization(momentum = 0.9)(x)
        x = LeakyReLU(alpha = 0.2)(x)
        x = Dropout(0.5)(x)
        return x
    
    # input image
    input_img = Input(shape=input_shape)
    
    x = Conv2D(32, kernel_size = (3, 3), strides = (1, 1), padding='same', 
               kernel_initializer = init)(input_img)
    x = BatchNormalization(momentum = 0.9)(x)
    x = LeakyReLU(alpha = 0.2)(x)
    x = Dropout(0.5)(x)

    # downsample to 56 x 56 x 64
    x = conv_block(x, 64, (2, 2))
    # downsample to 28 x 28 x 128
    x = conv_block(x, 128, (2, 2))
#     x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
#     x = BatchNormalization(momentum = 0.9)(x)
#     x = LeakyReLU(alpha = 0.2)(x)
#     x = Dropout(0.5)(x)
    # downsample to 14 x 14 x 256
    x = conv_block(x, 256, (2, 2))
    # downsample to 7 x 7 x 512
    x = conv_block(x, 512, (2, 2))
#     x = MaxPooling2D(pool_size = (2, 2), strides = (2, 2))(x)
#     x = BatchNormalization(momentum = 0.9)(x)
#     x = LeakyReLU(alpha = 0.2)(x)
#     x = Dropout(0.5)(x)
    
    # flatten layer
    features = Flatten()(x)

    # binary classifier, image fake or real
    fake = Dense(1, activation='sigmoid', name='generation')(features)

    # multi-class classifier, image digit class
    aux = Dense(3, activation='softmax', name='auxiliary')(features)

    model = Model(inputs = input_img,  outputs = [fake, aux])

    return model

In [11]:
d = discriminator(input_shape=(112, 112, 3))
d.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_4 (Conv2D)              (None, 112, 112, 32  896         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 112, 112, 32  128        ['conv2d_4[0][0]']               
 rmalization)                   )                                                           

/home/giorgio/miniconda3/envs/DL38/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [9]:
def create_discriminator():
    inputs = Input(shape=(112, 112, 1))
    leaky = tf.keras.layers.LeakyReLU(0.2)

    conv1 = Conv2D(32, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Dropout(0.2)(conv1)

    conv1 = Conv2D(32, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Dropout(0.2)(conv2)

    conv2 = Conv2D(64, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Dropout(0.2)(conv3)

    conv3 = Conv2D(128, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Dropout(0.2)(conv4)

    conv4 = Conv2D(256, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Dropout(0.2)(conv5)

    conv5 = Conv2D(512, 3, activation=leaky, padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    gap1 = GlobalAveragePooling2D()(drop5)
    fc1 = Dense(128)(gap1)
            
    output_disc = Dense(1, activation="linear")(fc1)
    output_class = Dense(3, activation="linear")(fc1)

    model = Model(inputs=inputs, outputs=[output_disc, output_class])
    
    return model

In [9]:
create_discriminator().summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 112, 112, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 112, 112, 32  320         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_10 (Dropout)           (None, 112, 112, 32  0           ['conv2d_10[0][0]']              
                                )                                                           

# Generator

In [4]:
def generator(latent_dim = 100, n_classes = 3):
    init = RandomNormal(mean = 0.0, stddev = 0.02)
  
    # Input 1: class label input
    label_input = Input(shape = (1,))
    print(label_input.shape)
    y = Embedding(n_classes, 100)(label_input)
    print('Embedding Layer: ', y.shape)
    n_nodes = 7 * 7
    y = Dense(n_nodes, kernel_initializer = init)(y)
    print('Dense 1: ', y.shape)
    y = Reshape((7, 7 ,1))(y)
    print('reshape(final y shape): ', y.shape)

    # Input 2: generator noise input
    generator_input = Input(shape = (latent_dim,))
    n_nodes = 1024 * 7 * 7
    gen = Dense(n_nodes, kernel_initializer = init)(generator_input)
    gen = Activation('relu')(gen)
    gen = Reshape((7, 7, 1024))(gen)
    print('Generator noise input: ', gen.shape)
    # Concatenate both the inputs
    merge = Concatenate()([gen, y])
    print('Concatenate(generator noise input and y: ', merge.shape)

    # (None, 7, 7, 1024) --> (None, 14, 14, 512)
    gen = Conv2DTranspose(512, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = init)(merge)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print("(None, 7, 7, 1024) -> (None, 14, 14, 512): ", gen.shape)

    # (None, 14, 14, 512)  --> (None, 28, 28, 256)
    gen = Conv2DTranspose(256, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = init)(gen)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print('(None, 14, 14, 512) -> (None, 28, 28, 256): ', gen.shape)

    # (None, 28, 28, 256) --> (None, 56, 56, 128)
    gen = Conv2DTranspose(128, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = init)(gen)
    gen = BatchNormalization(momentum = 0.9)(gen)
    gen = Activation("relu")(gen)
    print('(None, 28, 28, 256) -> (None, 56, 56, 128): ', gen.shape)

    # (None, 56, 56, 128) --> (None, 112, 112, 3)
    gen = Conv2DTranspose(3, kernel_size = (5, 5), strides = (2, 2), padding = "same", kernel_initializer = init)(gen)
    out_layer = Activation("tanh")(gen)
    print("(None, 56, 56, 128) -> (None, 112, 112, 3): ", out_layer.shape)
    
    model = Model(inputs = [generator_input, label_input], outputs = out_layer)
    return model

In [7]:
g = generator(latent_dim = 100, n_classes = 3)
g.summary()

(None, 1)
Embedding Layer:  (None, 1, 100)
Dense 1:  (None, 1, 49)
reshape(final y shape):  (None, 7, 7, 1)
Generator noise input:  (None, 7, 7, 1024)
Concatenate(generator noise input and y:  (None, 7, 7, 1025)
(None, 7, 7, 1024) -> (None, 14, 14, 512):  (None, 14, 14, 512)
(None, 14, 14, 512) -> (None, 28, 28, 256):  (None, 28, 28, 256)
(None, 28, 28, 256) -> (None, 56, 56, 128):  (None, 56, 56, 128)
(None, 56, 56, 128) -> (None, 112, 112, 3):  (None, 112, 112, 3)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                 

# AC-cGAN model building

In [6]:
adam_lr = 0.0002
adam_beta_1 = 0.5

def define_gan(latent_dim = 100):
    # build the discriminator
    dis = discriminator()
    dis.summary()
    dis.compile(
        optimizer=Adam(learning_rate=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )

    # build the generator
    gen = generator(latent_dim)
    gen.summary()
    gen.compile(optimizer=Adam(learning_rate=adam_lr, beta_1=adam_beta_1),
                      loss='binary_crossentropy')

    # Inputs
    latent = Input(shape=(latent_dim, ), name='latent_noise')
    image_class = Input(shape=(1,), name='image_class')
    print(image_class.dtype)
    # Get a fake image
    fake_img = gen([latent, image_class])
    print('fake image: ', fake_img.shape)
    # Only train generator in combined model
    dis.trainable = False
    fake, aux = dis(fake_img)
    combined = Model(inputs=[latent, image_class],
                            outputs=[fake, aux],
                            name='ACGAN')

    combined.compile(
        optimizer=Adam(learning_rate=adam_lr, beta_1=adam_beta_1),
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
    )
    
    combined.summary()
    return combined, dis, gen

In [9]:
gan_model = define_gan(latent_dim=100)

/home/giorgio/miniconda3/envs/DL38/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


(None, 1)
Embedding Layer:  (None, 1, 100)
Dense 1:  (None, 1, 49)
reshape(final y shape):  (None, 7, 7, 1)
Generator noise input:  (None, 7, 7, 1024)
Concatenate(generator noise input and y:  (None, 7, 7, 1025)
(None, 7, 7, 1024) -> (None, 14, 14, 512):  (None, 14, 14, 512)
(None, 14, 14, 512) -> (None, 28, 28, 256):  (None, 28, 28, 256)
(None, 28, 28, 256) -> (None, 56, 56, 128):  (None, 56, 56, 128)
(None, 56, 56, 128) -> (None, 112, 112, 3):  (None, 112, 112, 3)
<dtype: 'float32'>
fake image:  (None, 112, 112, 3)
Model: "ACGAN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 latent_noise (InputLayer)      [(None, 100)]        0           []                               
                                                                                                  
 image_class (InputLayer)       [(None, 1)]          0           [

# AC-cGAN Trainer

## Tools

In [7]:
def label_smoothing(vector, max_dev = 0.2):
        d = max_dev * np.random.rand(vector.shape[0],vector.shape[1])
        if vector[0][0] == 0:
            return vector + d
        else:
            return vector - d
          
valid_o = np.ones((batch_size, 1))
fake_o = np.zeros((batch_size, 1))

# Function for printing the logs after every epoch

def print_logs(metrics_names, train_history, test_history):

    print('{0:<22s} | {1:4s} | {2:15s} | {3:5s}'.format(
        'component', *metrics_names))
    print('-' * 65)

    ROW_FMT = '{0:<22s} | {1:<4.2f} | {2:<15.2f} | {3:<5.2f}'
    print(ROW_FMT.format('generator (train)',
                         *train_history['generator'][-1]))
    print(ROW_FMT.format('generator (test)',
                         *test_history['generator'][-1]))
    print(ROW_FMT.format('discriminator (train)',
                         *train_history['discriminator'][-1]))
    print(ROW_FMT.format('discriminator (test)',
                         *test_history['discriminator'][-1]))

# Function to generate a batch of noise and label 

def generate_batch_noise_and_labels(batch_size, latent_dim, n_classes = 3):

    # generate a new batch of noise
    noise = np.random.uniform(-1, 1, (batch_size, latent_dim))

    # sample some labels
    sampled_labels = np.random.randint(0, n_classes, batch_size)

    return noise, sampled_labels

In [8]:
np.random.randint(0, 3, batch_size)

array([1, 0, 2, 0, 0, 1, 2, 2, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 1, 1, 2, 0,
       0, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2])

In [ ]:

# Function to generate the loss graph using the saved weights

import pandas as pd
path = './AC-cGAN/weights/acgan_history.pkl'
path.encode('utf-8').strip()
hist = pickle.load(open(path, 'rb'))

for p in ['train', 'test']:
    for g in ['discriminator', 'generator']:
        hist[p][g] = pd.DataFrame(hist[p][g], columns=['loss', 'generation_loss', 'auxiliary_loss'])
        plt.plot(hist[p][g]['generation_loss'], label='{} ({})'.format(g, p))

# get the NE and show as an equilibrium point
plt.hlines(-np.log(0.5), 0, hist[p][g]['generation_loss'].shape[0], label='Nash Equilibrium')
plt.legend()
plt.title(r'$L_s$ (generation loss) per Epoch')
plt.xlabel('Epoch')
plt.ylabel(r'$L_s$')

## Trainer

In [9]:
nb_epochs = 1200
batch_size = 36
latent_dim = 100

train_history = defaultdict(list)
test_history = defaultdict(list)

In [10]:
combined, dis, gen = define_gan(latent_dim = 100)

2023-02-06 19:58:28.400204: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-06 19:58:28.402658: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-06 19:58:28.402691: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-06 19:58:28.403414: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them i

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 112, 112, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [71]:
print(len(valid.ravel()))

36


In [44]:
noise, sampled_labels = generate_batch_noise_and_labels(batch_size, latent_dim, n_classes = 3)
print(sampled_labels.reshape((-1, 1))[1:5])
generated_images = gen.predict([noise, sampled_labels.reshape((-1, 1))], verbose=1)
#print(generated_images[1])
print(len(generated_images))
print(generated_images.shape)
X = np.concatenate((batchX, generated_images))
print(X.shape)

valid = label_smoothing(vector = valid_o, max_dev = 0.2)
#print(valid)
print(valid.shape)
fake = label_smoothing(vector = fake_o, max_dev = 0.2)
print(fake.shape)

y = np.concatenate((valid, fake), axis = 0)
print(y.shape)
aux_y = np.concatenate((batchy, sampled_labels), axis=0)
aux_y = np.reshape(aux_y, (-1, 1))
print(aux_y.shape)
print(dis.predict(X))
print(dis.train_on_batch(X, [y, aux_y]))

#print(y)
#print(aux_y)
print(len([y, aux_y]))

[[2]
 [0]
 [0]
 [2]]
2/2 [==============================] - 0s 400ms/step
36
(36, 112, 112, 3)
(72, 112, 112, 3)
(36, 1)
(36, 1)
(72, 1)
(72, 1)
3/3 [==============================] - 0s 8ms/step
[array([[0.46632695],
       [0.44227803],
       [0.47825062],
       [0.47187778],
       [0.46730426],
       [0.49230516],
       [0.44712755],
       [0.46967837],
       [0.49164385],
       [0.45876145],
       [0.46461704],
       [0.5041681 ],
       [0.44422537],
       [0.4657744 ],
       [0.48983222],
       [0.4625748 ],
       [0.5186298 ],
       [0.4690823 ],
       [0.49717858],
       [0.48496255],
       [0.44700554],
       [0.4461117 ],
       [0.485333  ],
       [0.46369112],
       [0.46807384],
       [0.5257085 ],
       [0.47300136],
       [0.47402355],
       [0.49519083],
       [0.4526591 ],
       [0.48429024],
       [0.4707228 ],
       [0.4920724 ],
       [0.45337746],
       [0.4745044 ],
       [0.4589809 ],
       [0.50000334],
       [0.49999928],
     

2023-02-06 19:51:54.334567: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/dropout_25/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


In [28]:
np.reshape(aux_y, (-1, 1)).shape

(72, 1)

In [15]:
for epoch in range(nb_epochs):
    print('Epoch {} of {}'.format(epoch + 1, nb_epochs))

    nb_batches = int(1332/batch_size)
    progress_bar = Progbar(target=nb_batches)

    epoch_gen_loss = []
    epoch_disc_loss = []
    gen_test_loss = []
    disc_test_loss = []

    for index, (image_batch, label_batch) in zip(range(nb_batches), train_data):
        #print("t1")
        image_batch = image_batch * (1. / 127.5) - 1

        progress_bar.update(index)
        #print("t2")
        noise, sampled_labels = generate_batch_noise_and_labels(batch_size, latent_dim)
        #print("t3")
        generated_images = gen.predict([noise, sampled_labels.reshape((-1, 1))],  verbose=0)
        #print("t4")
        print(len(generated_images))
        print(len(image_batch))
        X = np.concatenate((image_batch, generated_images))
        print(len(X))
        #print("t5")
        valid = label_smoothing(vector = valid_o, max_dev = 0.2)
        fake = label_smoothing(vector = fake_o, max_dev = 0.2)
        #print("t6")
        y = np.concatenate((valid, fake), axis = 0)
        aux_y = np.concatenate((label_batch, sampled_labels), axis=0)
        #print("t7")
        #print(dis.train_on_batch(X, [y, aux_y]))
        #print("t7.5")
        epoch_disc_loss.append(dis.train_on_batch(X, [y, aux_y]))
        #print("t8")
        noise, sampled_labels = generate_batch_noise_and_labels(2 * batch_size, latent_dim)
                
        trick = np.ones(2 * batch_size)
        #print("t5")
        epoch_gen_loss.append(combined.train_on_batch(
            [noise, sampled_labels.reshape((-1, 1))], [trick, sampled_labels]))
        #print("t6")
        
    for test_image_batch, test_label_batch in test_data:
                
        print('\nTesting for epoch {}:'.format(epoch + 1))

        test_image_batch = test_image_batch * (1. / 127.5) - 1

        nb_train = test_image_batch.shape[0]
        #nb_test = test_label_batch

        noise, sampled_labels = generate_batch_noise_and_labels(nb_train, latent_dim)

        generated_images = gen.predict(
            [noise, sampled_labels.reshape((-1, 1))], verbose=False
        )  
            
        X = np.concatenate((test_image_batch, generated_images))
        y = np.array([1] * nb_train + [0] * nb_train)
        aux_y = np.concatenate((test_label_batch, sampled_labels), axis=0)
        
        #print('1')
        
        test_discriminator_loss = dis.evaluate(X, [y, aux_y], verbose = False)

        #print('2')
        
        disc_test_loss.append(test_discriminator_loss)

        noise, sampled_labels = generate_batch_noise_and_labels(2 * nb_train, latent_dim)


        trick = np.ones(2 * nb_train)

        test_generator_loss = combined.evaluate(
            [noise, sampled_labels.reshape((-1, 1))],
            [trick, sampled_labels], verbose = False
        )

        gen_test_loss.append(test_generator_loss)

        break

    discriminator_train_loss = np.mean(np.array(epoch_disc_loss), axis=0)

    generator_train_loss = np.mean(np.array(epoch_gen_loss), axis=0)

    generator_test_loss = np.mean(np.array(gen_test_loss), axis=0)

    discriminator_test_loss = np.mean(np.array(disc_test_loss), axis=0)

    train_history['generator'].append(generator_train_loss)
    train_history['discriminator'].append(discriminator_train_loss)

    test_history['generator'].append(generator_test_loss)
    test_history['discriminator'].append(discriminator_test_loss)

    print_logs(dis.metrics_names, train_history, test_history)

    # gen.save_weights(
    #     './AC-cGAN/parameters/params_generator_epoch_{0:03d}.hdf5'.format(epoch), True)
    # dis.save_weights(
    #     './AC-cGAN/parameters/params_discriminator_epoch_{0:03d}.hdf5'.format(epoch), True)

pickle.dump({'train': train_history, 'test':test_history},
            open('./AC-cGAN/weights/acgan_history.pkl', 'wb'))

Epoch 1 of 1200
 0/37 [..............................] - ETA: 0s36
36
72
 1/37 [..............................] - ETA: 35s36
36
72
 2/37 [>.............................] - ETA: 17s36
36
72
 3/37 [=>............................] - ETA: 17s36
36
72
 4/37 [==>...........................] - ETA: 16s36
36
72
 5/37 [===>..........................] - ETA: 15s36
36
72
 6/37 [===>..........................] - ETA: 15s36
36
72
 7/37 [====>.........................] - ETA: 14s36
36
72
 8/37 [=====>........................] - ETA: 14s36
36
72
 9/37 [======>.......................] - ETA: 13s36
36
72
10/37 [=======>......................] - ETA: 13s36
36
72
11/37 [=======>......................] - ETA: 12s36
36
72
12/37 [========>.....................] - ETA: 12s36
36
72
13/37 [=========>....................] - ETA: 11s36
36
72
14/37 [==========>...................] - ETA: 11s36
36
72
15/37 [===========>..................] - ETA: 10s36
36
72
16/37 [===========>..................] - ETA: 10s36
36
72

ValueError: Data cardinality is ambiguous:
  x sizes: 41
  y sizes: 72, 41
Make sure all arrays contain the same number of samples.

# Test Generator

In [7]:
g_test = generator(latent_dim = 100, n_classes = 3)
g_test.summary()

(None, 1)
Embedding Layer:  (None, 1, 100)
Dense 1:  (None, 1, 49)
reshape(final y shape):  (None, 7, 7, 1)
Generator noise input:  (None, 7, 7, 1024)
Concatenate(generator noise input and y:  (None, 7, 7, 1025)
(None, 7, 7, 1024) -> (None, 14, 14, 512):  (None, 14, 14, 512)
(None, 14, 14, 512) -> (None, 28, 28, 256):  (None, 28, 28, 256)
(None, 28, 28, 256) -> (None, 56, 56, 128):  (None, 56, 56, 128)
(None, 56, 56, 128) -> (None, 112, 112, 3):  (None, 112, 112, 3)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                 

/home/giorgio/miniconda3/envs/DL38/lib/python3.8/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [18]:
adam_lr = 0.0002
adam_beta_1 = 0.5
nb_epochs = 1200
batch_size = 36
latent_dim = 100

train_history = defaultdict(list)
test_history = defaultdict(list)

g.compile(optimizer=Adam(learning_rate=adam_lr, beta_1=adam_beta_1),
                      loss='binary_crossentropy')

noise, sampled_labels = generate_batch_noise_and_labels(batch_size, latent_dim)
print(len(noise))
print(len(sampled_labels))
generated_images = g.predict([noise, sampled_labels.reshape((-1, 1))], verbose=0)
print(generated_images[1].shape)

36
36
(112, 112, 3)
